In [220]:
import json
split = "train"
with open(f'../data/duckie_sim_{split}.json') as json_file:
    sim_data = json.load(json_file)
with open(f'../data/duckie_js_{split}.json') as json_file:
    js_data = json.load(json_file)

In [221]:
!mkdir -p merged_dataset
!mkdir -p merged_dataset/rgb

In [222]:
sim_images = sim_data["images"]

In [223]:
js_images = js_data["images"]

In [225]:
len(js_images), len(sim_images)

(9000, 50000)

In [226]:
import shutil
from tqdm import tqdm
for image in tqdm(js_images):
    filename = image["file_name"]
    shutil.copy(f"../data/dt_segmentation_dataset/new/wo_bezier/rgb_orig/{filename}", f"merged_dataset/rgb/js_{filename}")
    

100%|██████████| 9000/9000 [00:00<00:00, 12800.09it/s]


In [227]:
for image in tqdm(sim_images):
    filename = image["file_name"]
    shutil.copy(f"../data/dt_segmentation_dataset/sim/rgb_160_120//{filename}", f"merged_dataset/rgb/sim_{filename}")
    

100%|██████████| 50000/50000 [00:03<00:00, 14290.58it/s]


In [230]:
#Fresh data
with open(f'../data/duckie_sim_{split}.json') as json_file:
    sim_data = json.load(json_file)
with open(f'../data/duckie_js_{split}.json') as json_file:
    js_data = json.load(json_file)
    


In [231]:
merged_images=[]
merged_id = 0
js_image_mapping = {}
sim_image_mapping = {}
for image in js_data["images"]:
    new_image = image.copy()
    new_image["file_name"] = "js_" + image["file_name"]
    js_image_mapping[image["id"]] = merged_id
    new_image["id"] = merged_id
    merged_id+=1
    merged_images.append(new_image)


In [232]:
merged_images[0]

{'file_name': 'js_8313.jpg', 'height': 120, 'width': 160, 'id': 0}

In [233]:
for image in sim_data["images"]:
    new_image = image.copy()
    new_image["file_name"] = "sim_" + image["file_name"]
    #print(image["id"])
    sim_image_mapping[image["id"]] = merged_id
    new_image["id"] = merged_id
    merged_id+=1
    merged_images.append(new_image)

In [234]:
len(merged_images), len(sim_images), len(js_images)

(59000, 50000, 9000)

In [236]:
#js_data["annotations"][0]

In [241]:
merged_annotations = []
annotation_id = 0
for annotation in js_data["annotations"]:
    new_annotation = annotation.copy()
    new_annotation["image_id"] = js_image_mapping[annotation["image_id"]]
    new_annotation["id"] = annotation_id
    annotation_id+=1
    merged_annotations.append(new_annotation)

print(len(merged_annotations))

109289


In [242]:
for annotation in sim_data["annotations"]:
    new_annotation = annotation.copy()
    new_annotation["image_id"] = sim_image_mapping[annotation["image_id"]]
    new_annotation["id"] = annotation_id
    annotation_id+=1
    merged_annotations.append(new_annotation)
print(len(merged_annotations))

799010


In [244]:
data={"categories":sim_data["categories"], "images":merged_images,"annotations":merged_annotations}
with open(f"../data/duckie_merged_{split}.json","w") as f:
    json.dump(data,f)

In [245]:
import glob

In [246]:
files = glob.glob("merged_dataset/rgb/*.jpg")
import shutil, os

In [247]:
for file in tqdm(files):
    basename = os.path.basename(file)
    shutil.copy(file, f"../data/dt_segmentation_dataset/merged/rgb/{basename}")
    

100%|██████████| 69559/69559 [00:04<00:00, 14026.22it/s]
